# cifar110-PN

- slide: http://www.ms.k.u-tokyo.ac.jp/~gang/paper/niu_ibisml030.pdf
- ipynb: https://github.com/JihongJu/deep-pu-learning/blob/master/experiments/cifar/cifar110-PN.ipynb

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import" data-toc-modified-id="Import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import</a></span></li><li><span><a href="#CIFAR110" data-toc-modified-id="CIFAR110-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>CIFAR110</a></span><ul class="toc-item"><li><span><a href="#Load-Dataset" data-toc-modified-id="Load-Dataset-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load Dataset</a></span></li><li><span><a href="#[Def]-normalize" data-toc-modified-id="[Def]-normalize-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>[Def] normalize</a></span></li><li><span><a href="#[Def]-synthesize_pu_labels" data-toc-modified-id="[Def]-synthesize_pu_labels-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>[Def] synthesize_pu_labels</a></span></li><li><span><a href="#[Def]-synthesize_pn_labels" data-toc-modified-id="[Def]-synthesize_pn_labels-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>[Def] synthesize_pn_labels</a></span></li><li><span><a href="#[Def]-VGG8" data-toc-modified-id="[Def]-VGG8-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>[Def] VGG8</a></span></li><li><span><a href="#Fit" data-toc-modified-id="Fit-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Fit</a></span></li></ul></li></ul></div>

## Import

In [2]:
from __future__ import (print_function, absolute_import)

import os
#-------------------------------------------------------------------------------
#  OPTIONAL
#  set a GPU viewed by TensorFlow
##---------------------------------------
##  - https://stackoverflow.com/questions/37893755/tensorflow-set-cuda-visible-devices-within-jupyter
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"
##---------------------------------------

import re

import numpy as np
print("numpy ver: {}".format(np.__version__))

import pandas as pd
print("pandas ver: {}".format(pd.__version__))

import keras
print("keras ver: {}".format(keras.__version__))
import keras.backend as K
from keras import datasets
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.callbacks import (
    ReduceLROnPlateau,
    LearningRateScheduler,
    CSVLogger,
    EarlyStopping,
    ModelCheckpoint)

import tensorflow as tf
print("tensorflow ver: {}".format(tf.__version__))
# You can double check that you have the correct devices visible to TF
#   - https://stackoverflow.com/questions/37893755/tensorflow-set-cuda-visible-devices-within-jupyter
from tensorflow.python.client import device_lib
print("""
________________________________________
Visible GPUs from TensorFlow
________________________________________""")
for _device in device_lib.list_local_devices():
    match = re.search(pattern=r'name: "/device:(?P<name>[A-Z]{3}):(?P<device_num>\d{1})*',
                      string=str(_device))
    if match is None:
        print("Not Match")
        continue
    if match.group("name") == "CPU":
        name, device_num = match.group("name", "device_num")
        print()
        print("({}:{})".format(name, device_num))
        continue
    name, device_num = match.group("name", "device_num")
    match = re.search(pattern=r'.*pci bus id: (?P<pci_bus_id>\d{4}:\d{2}:\d{2}.\d{1}).*',
                      string=str(_device))
    if match is None:
        print("No GPUs")
        continue
    print("({}:{}: pci_bus_id: {})".format(name, device_num, match.group("pci_bus_id")))
print("________________________________________")

global _SESSION
config = tf.ConfigProto(allow_soft_placement=True,
                        log_device_placement=True)
config.gpu_options.allow_growth = True
_SESSION = tf.Session(config=config)
K.set_session(_SESSION)


import keras_tqdm
from keras_tqdm import TQDMNotebookCallback
#print("keras_tqdm ver: {}".format(keras_tqdm.__version__))

numpy ver: 1.15.0
pandas ver: 0.23.3


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


keras ver: 2.2.2
tensorflow ver: 1.9.0

________________________________________
Visible GPUs from TensorFlow
________________________________________

(CPU:0)
(GPU:0: pci_bus_id: 0000:02:00.0)
________________________________________


## CIFAR110

In [3]:
num_classes = 11
batch_size = 128
epochs = 200
data_augmentation = True
checkpoint = None
# checkpoint = 'model_checkpoint_cifar10_wide_resnet.h5'
title = 'cifar110_vgg8'
# title = 'cifar110_wide_resnet'

### Load Dataset

In [20]:
(x_train_10, y_train_10), (x_test_10, y_test_10) = datasets.cifar10.load_data()
(x_train_100, y_train_100), (x_test_100, y_test_100) = datasets.cifar100.load_data()
print((x_train_10.shape, y_train_10.shape), (x_test_10.shape, y_test_10.shape))
print((x_train_100.shape, y_train_100.shape), (x_test_100.shape, y_test_100.shape))

#--------------------------------------------------------------------------------
#  Chage Label
##----------------------------------------
##  - cifar10 のデータセットのラベルを一つずらす(+1)
##  - cifar100 のデータセットをすべて0にする
##    - cifar100 を unlabeled datset とする
y_train_10 = y_train_10 + 1
y_test_10 = y_test_10 + 1
y_train_100[...] = 0
y_test_100[...] = 0

((50000, 32, 32, 3), (50000, 1)) ((10000, 32, 32, 3), (10000, 1))
((50000, 32, 32, 3), (50000, 1)) ((10000, 32, 32, 3), (10000, 1))


In [5]:
x_train = np.concatenate((x_train_10, x_train_100), axis=0).astype('float32')
y_train = np.concatenate((y_train_10, y_train_100), axis=0).astype('int8')
x_test = np.concatenate((x_test_10, x_test_100), axis=0).astype('float32')
y_test = np.concatenate((y_test_10, y_test_100), axis=0).astype('int8')

x_train = x_train.astype(K.floatx())
x_test = x_test.astype(K.floatx())

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### [Def] normalize

In [6]:
def normalize(x, mean, std):
    """
    各チャネルの平均・標準偏差を用いて標準化
    """
    import copy
    x = copy.deepcopy(x)

    x -= np.array(object=mean, dtype=K.floatx())
    x /= np.array(object=std, dtype=K.floatx())
    return x

In [7]:
print(x_train.shape)

mean = x_train.mean(axis=(0, 1, 2))
print(mean)
std = x_train.std(axis=(0, 1, 2))
print(std)

x_train = normalize(x=x_train, mean=mean, std=std)
x_test = normalize(x=x_test, mean=mean, std=std)

(100000, 32, 32, 3)
[127.3043  123.50989 113.14998]
[65.66327 63.76426 68.59032]


In [8]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(100000, 32, 32, 3) (100000, 11)
(20000, 32, 32, 3) (20000, 11)


### [Def] synthesize_pu_labels

In [19]:
def synthesize_pu_labels(Y, pct_missings=None, random_state=None,
                         verbose=False):
    """
    Synthesize PU labels.
    reference:
      - https://github.com/JihongJu/deep-pu-learning/blob/master/pulearn/utils.py
      - https://github.com/JihongJu/deep-pu-learning/blob/d896ca838e9629d9d418fd36f1138b9084c97bea/pulearn/utils.py
    """
    # To PU
    if pct_missings is None:
        pct_missings = np.arange(0., 1 + 1e-8, 0.1)
    Y_pu = {}
    np.random.seed(random_state)
    n_samples = len(Y)
    for pct in pct_missings:
        y = np.argmax(Y, 1)  # unlabel が 1, 他は 0
        flip = np.random.rand(n_samples)
        y[(y != 0) & (flip < pct)] = 0
        Y_pu[pct] = np.eye(Y.shape[1])[y]
        if verbose is True:
            print('Positive (pct_missing={}):'.format(pct),
                  np.sum(np.argmax(Y, 1)>0), ' -> ', np.sum(y>0))
    return Y_pu

### [Def] synthesize_pn_labels

In [18]:
def synthesize_pn_labels(Y, pct_missings=None, random_state=None,
                         verbose=False):
    """
    Synthesize PN labels.
    reference:
      - https://github.com/JihongJu/deep-pu-learning/blob/master/pulearn/utils.py
      - https://github.com/JihongJu/deep-pu-learning/blob/d896ca838e9629d9d418fd36f1138b9084c97bea/pulearn/utils.py
    """
    # To PN
    if pct_missings is None:
        pct_missings = np.arange(0., 1 + 1e-8, 0.1)
    idx = {}
    np.random.seed(random_state)
    n_samples = len(Y)
    for pct in pct_missings:
        y = np.argmax(Y, 1)  # unlabel が 1, 他は 0
        flip = np.random.rand(n_samples)
        mask = flip < pct
        y[mask] = -1
        idx[pct] = np.where(np.invert(mask))[0]
        if verbose is True:
            print('Positive vs. Negative (pct={}):'.format(pct),
                  np.sum(y > 0), ' vs.', np.sum(y == 0))
    return idx

In [11]:
pct_missing = 0.5
y_train_pu = synthesize_pu_labels(y_train, random_state=None, verbose=True)

Positive (pct_missing=0.0): 50000  ->  50000
Positive (pct_missing=0.1): 50000  ->  44884
Positive (pct_missing=0.2): 50000  ->  40040
Positive (pct_missing=0.30000000000000004): 50000  ->  34933
Positive (pct_missing=0.4): 50000  ->  30039
Positive (pct_missing=0.5): 50000  ->  24978
Positive (pct_missing=0.6000000000000001): 50000  ->  20015
Positive (pct_missing=0.7000000000000001): 50000  ->  14944
Positive (pct_missing=0.8): 50000  ->  10051
Positive (pct_missing=0.9): 50000  ->  5070
Positive (pct_missing=1.0): 50000  ->  0


### [Def] VGG8

In [12]:
def VGG8(input_shape, num_classes, t_vec=None):
    from keras.models import Sequential
    from keras.layers import Dense, Dropout, Activation, Flatten, Lambda
    from keras.layers import Conv2D, MaxPooling2D
    from keras.layers.advanced_activations import LeakyReLU

    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal',
                                input_shape=input_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(512, kernel_initializer='he_normal'))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, kernel_initializer='he_normal'))
    # output translate (tensorflow only)
    if t_vec is not None:
        def translate(x):
            return x - t_vec
        model.add(Lambda(lambda x: translate(x)))
    model.add(Activation('softmax'))

    return model

In [13]:
if title == 'cifar110_vgg8':
    model = VGG8(input_shape=x_train.shape[1:], num_classes=num_classes)
elif title == 'cifar110_wide_resnet':
    model = WideResidualNetwork(depth=28, width=8, dropout_rate=0.5,
                                classes=num_classes, include_top=True,
                                weights=None)
if checkpoint is not None:
    model = load_model(checkpoint)
    
optimizer = keras.optimizers.Adam(lr=1e-4)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
__________

In [14]:
# Checkpoint
checkpointer = ModelCheckpoint(
    monitor='val_acc',
    filepath="model_checkpoint_{}_pu_{}.h5".format(title, pct_missing),
    verbose=1,
    save_best_only=True)

# csvlogger
csv_logger = CSVLogger(
    'csv_logger_{}_pu_{}.csv'.format(title, pct_missing))

# EarlyStopping
early_stopper = EarlyStopping(monitor='val_acc',
                              min_delta=0.001,
                              patience=50)
# Reduce lr with schedule
# def schedule(epoch):
#     lr = K.get_value(model.optimizer.lr)
#     if epoch in [60, 120, 160]:
#         lr = lr * np.sqrt(0.1)
#     return lr
# lr_scheduler = LearningRateScheduler(schedule)

# Reduce lr on plateau
lr_reducer = ReduceLROnPlateau(monitor='val_acc',
                               factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=20,
                               min_lr=1e-6)

### Fit

In [15]:
if not data_augmentation:
    print('No data augmentation applied.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              class_weight=class_weight,
              shuffle=True,
              callbacks=[csv_logger, checkpointer, early_stopper, lr_reducer]) 
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        # randomly rotate images in the range (degrees, 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train_pu[pct_missing],
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        callbacks=[checkpointer, early_stopper,
                                   lr_reducer, csv_logger,
                                   TQDMNotebookCallback()])
    model.save('{}_pu_{}.h5'.format(title, pct_missing))
    
    # Predict
    y_train_pred_enc = model.predict(x_train)
    y_test_pred_enc = model.predict(x_test)
    
    lst = []
    y_train_pred = np.argmax(y_train_pred_enc, axis=-1)
    y_train_true = np.argmax(y_train, axis=-1)
    y_train_label = np.argmax(y_train_pu[pct_missing], axis=-1)
    for y_pred, y_true, y_label in zip(y_train_pred, y_train_true, y_train_label):
        lst.append(dict(y_pred=y_pred, y_true=y_true, y_label=y_label, split='train'))

    y_test_pred = np.argmax(y_test_pred_enc, axis=-1)
    y_test_true = np.argmax(y_test, axis=-1)
    for y_pred, y_true in zip(y_test_pred, y_test_true):
        lst.append(dict(y_pred=y_pred, y_true=y_true, y_label=y_true, split='test'))
    
    df = pd.DataFrame(lst)
    df.to_csv('prediction_{}_pu_{}.csv'.format(title, pct_missing), index=False, mode='a')

Using real-time data augmentation.


Epoch 1/200


781/781 [==============================] - 27s 35ms/step - loss: 1.2390 - acc: 0.7439 - val_loss: 1.5849 - val_acc: 0.4999

Epoch 00001: val_acc improved from -inf to 0.49990, saving model to model_checkpoint_cifar110_vgg8_pu_0.5.h5
Epoch 2/200


781/781 [==============================] - 27s 34ms/step - loss: 1.0364 - acc: 0.7491 - val_loss: 1.4716 - val_acc: 0.5112

Epoch 00002: val_acc improved from 0.49990 to 0.51125, saving model to model_checkpoint_cifar110_vgg8_pu_0.5.h5
Epoch 3/200


781/781 [==============================] - 27s 34ms/step - loss: 0.9800 - acc: 0.7487 - val_loss: 1.4278 - val_acc: 0.5086

Epoch 00003: val_acc did not improve from 0.51125
Epoch 4/200


...

781/781 [==============================] - 27s 34ms/step - loss: 0.5843 - acc: 0.7606 - val_loss: 0.8708 - val_acc: 0.6446

Epoch 00125: val_acc did not improve from 0.65115
Epoch 126/200


781/781 [==============================] - 26s 34ms/step - loss: 0.5849 - acc: 0.7610 - val_loss: 0.8652 - val_acc: 0.6448

Epoch 00126: val_acc did not improve from 0.65115
Epoch 127/200


781/781 [==============================] - 26s 34ms/step - loss: 0.5843 - acc: 0.7608 - val_loss: 0.8638 - val_acc: 0.6478

Epoch 00127: val_acc did not improve from 0.65115
Epoch 128/200


781/781 [==============================] - 26s 34ms/step - loss: 0.5818 - acc: 0.7626 - val_loss: 0.8680 - val_acc: 0.6452

Epoch 00128: val_acc did not improve from 0.65115
Epoch 129/200


781/781 [==============================] - 26s 34ms/step - loss: 0.5827 - acc: 0.7607 - val_loss: 0.8656 - val_acc: 0.6472

Epoch 00129: val_acc did not improve from 0.65115

